In [ ]:

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

from tabulate import tabulate

print("--")


#  Setup stuff: Connectivity

In [ ]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [ ]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


# Create: Vertices/nodes, edges ..

In [ ]:

#  Vertices/Nodes ..

df_orders        = pd.DataFrame([
       #
   [101  , "Rayees Order"        ,    0,  "Order"     ],
   [102  , "Andrews Order"       ,    0,  "Order"     ],
   [103  , "Gregs Order"         ,    0,  "Order"     ],
       #
   ], columns = ["id", "order_name", "total", "LABEL"])
      #
print(tabulate(df_orders, headers='keys', tablefmt='psql'))


df_products      = pd.DataFrame([
       #
   [401  , "Apples"              ,   2,  "Product"   ],
   [402  , "Carrots"             ,  15,  "Product"   ],
   [403  , "Pears"               ,   7,  "Product"   ],
       #
   ], columns = ["id", "product_name", "unit_price", "LABEL"])
      #
print(tabulate(df_products, headers='keys', tablefmt='psql'))



In [ ]:

#  Create the edge ..

df_purchased = pd.DataFrame([
      #
   [ 101, 401 ,"CONTAINS"],
   [ 101, 402 ,"CONTAINS"],
   [ 101, 403 ,"CONTAINS"],
      #
   [ 102, 401 ,"CONTAINS"],
   [ 102, 401 ,"CONTAINS"],
   [ 102, 403 ,"CONTAINS"],
      #
   [ 103, 403 ,"CONTAINS"],
   [ 103, 403 ,"CONTAINS"],
   [ 103, 403 ,"CONTAINS"],
   [ 103, 403 ,"CONTAINS"],
      #
   ], columns = ["START_ID", "END_ID", "TYPE"])

print(tabulate(df_purchased, headers='keys', tablefmt='psql'))


In [ ]:


with import_data.DataFrameImporter(my_graph) as df_importer:
    
   df_importer.nodes_dataframe(
      df_orders[[
         "id", "order_name", "total", "LABEL"
      ]],
      id_column  = "id",
      id_space   = "Orders"
      )
    
   df_importer.nodes_dataframe(
      df_products[[
         "id", "product_name", "unit_price", "LABEL"
      ]],
      id_column  = "id",
      id_space   = "Products"
      )
    
   df_importer.edges_dataframe(
      df_purchased[[
         "START_ID", "END_ID", "TYPE"]],
      source_id_space      = "Orders",
      destination_id_space = "Products",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "RELATES_TO"
      )
   df_importer.edges_dataframe(
      df_purchased[[
         "START_ID", "END_ID", "TYPE"]],
      source_id_space      = "Products",
      destination_id_space = "Orders",
      source_column        = "END_ID",
      destination_column   = "START_ID",
      type                 = "RELATES_TO"
      )

   df_importer.insert()

print("--")



In [ ]:

#  What's in the vertices/nodes ..

l_query  = """
   MATCH ( n ) 
   RETURN n
   """.format()

l_return = my_graph.query(l_query)

print(tabulate(l_result, headers='keys', tablefmt='psql'))


In [57]:
#  What's in the graph .. 

l_result = my_graph.query("""

   MATCH (n) - [r] -> (m)
   RETURN n, r, m
   
   """, contextualize=True)

l_result.view()


          0/? [?op/s]

          0/? [?op/s]

#  Finally now; Why we're here ..

In [56]:

#  The traversal we are trying to run ..

l_result = my_graph.query("""

   //  Just fetching, works
   //
   MATCH (n: Orders) - [r :CONTAINS ] -> (m: Product)
   RETURN *
   // RETURN n, sum(m.unit_price) AS x_cost
   
   
   
   //  Closest to original question, throws error
   //
   // MATCH (n: Order) - [r :CONTAINS ] -> (m: Product)
   // WITH n, sum(m.unit_price) AS x_cost
   // SET Order.total = x_cost
   //
   //  OperationError: backtrace (QueryClient.cpp:391): unable to translate openCypher parse tree to internal IR (OpGraph.cpp:226):
   //  Syntax error: The query cannot be recognized by openCypher. (CypherOpGraphBuilder.cpp:63): TCK = SyntaxError:UnexpectedSyntax
   //  Katana = SyntaxError:ParserError: TCK = SyntaxError:UnexpectedSyntax
   //  Katana = SyntaxError:ParserError
   
   //  This also errors ..
   //
   // MATCH (n: Order) - [r :CONTAINS ] -> (m: Product)
   // WITH sum(m.unit_price) AS x_cost
   // SET Order.total = x_cost
   
   """, contextualize=True)

l_result.view()


#  Exact, original syntax ..
#
#  MATCH (loan)-[:hasLoanSnapshot]->(snapshot)
#  WITH loan, sum(snapshot.loan_snapshot_flag) as loan_snapshots
#  SET loan.loan_flag = loan_snapshots



          0/? [?op/s]